# Data Extraction

In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
%matplotlib inline  

In [9]:
import tweepy
import csv

In [10]:
# Authentication and setup for Tweepy

consumer_key = 'aQzHCKeyTaBKwqJaaiwda4bkO'
consumer_secret = 'JLRfDVG7k8GbbkkOZofL6Al69P2Ov1xhWRBAjFNFR4FwSKoXDj'
access_token = '3125452793-u04kuuehiQIPh94QUelRV6RREMfOxoDBq7oznT7'
access_token_secret = 'xcejx5JhpsvRvsWylOjXbeKyUce8AjxgsmFz54v5qdvrh'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True )

In [22]:
handles = pd.read_csv('TwitterHandles.csv')

In [34]:
# Function which, given a set of tweets from Tweepy API, will write them to a csv file

def write_tweets_to_csv(tweets, csvWriter, political_affiliation):
    
    for tweet in tweets:
        csvWriter.writerow([tweet.user.screen_name, tweet.full_text.encode('utf-8'), political_affiliation])
        

In [78]:
# Function for retrieving latest tweets of a given username

def get_user_tweets(username, tweet_count = 200): 
    
    tweets = api.user_timeline(screen_name = username, count = tweet_count, 
                          include_rts = False, tweet_mode='extended')
    return tweets
    

In [36]:
# Main function for recording tweets from an account

def record_tweets(username, csvWriter, politic):
    
    tweets = get_user_tweets(username)
    
    write_tweets_to_csv(tweets, csvWriter, politic)
    

In [37]:
# Given a dataframe of politician usernames will c

def get_tweets_from_dataframe(dataframe, csvWriter):
    
    for index, row in dataframe.iterrows():
        
        record_tweets(row['TwitterHandle'], csvWriter, row['Party'])

In [131]:
# Initialize a csv file
csvFile = open('tweets_final3.csv', 'a')

#Use csv Writer
csvWriter = csv.writer(csvFile)

In [ ]:
get_tweets_from_dataframe(handles, csvWriter)

In [133]:
# Load the csv into a dataframe

tweets_df1 = pd.read_csv('tweets_final3.csv', names = ["username", "tweets", "politics"])

# Data Cleaning

In [207]:
# Various methods for cleaning tweet strings to make them better suited for sentiment analysis

def remove_emojis(string):
    
    corrected = re.sub(r'\\x[\w+]{2}', '', string)
    
    return corrected

def correct_apostrophes(string):
    
    corrected = re.sub(r'\\xe2\\x80\\x99', '\'', string)
    
    return corrected

def remove_byte_encoding(string):
    return string[2:]

def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet)

In [221]:
def clean_string(string):
    
    #temp = remove_byte_encoding(string)
    temp = correct_apostrophes(string)
    temp = remove_emojis(temp)
    final = clean_tweet(temp)
    
    return final

In [214]:
def clean_dataframe_tweets(dataframe):
    
    new_strings = []
    
    for index, row in dataframe.iterrows():
        new_strings.append(clean_string(row['tweets_raw']))
    
    return new_strings

In [222]:
cleaned = clean_dataframe_tweets(tweets_df)

In [223]:
tweets_df['tweets_clean'] = cleaned

# Feature Engineering

### Sentiments, hashtags, and mentions

In [140]:
from textblob import TextBlob

In [101]:
# Extracts all hashtags from a string. Useful for getting hashtags from tweets

def extract_hashtags(string):
    hashtags = re.findall(r"#(\w+)", string)
    
    return hashtags


# Iterates over a dataframe and extracts all hashtags

def find_hashtags(dataframe):
    
    h_list = []
    
    for index, row in dataframe.iterrows():
        
        hashtags = extract_hashtags(row['tweets_raw'])
        
        h_list.append(hashtags)
    
    return h_list

In [167]:
# Extracts all hashtags from a string. Useful for getting hashtags from tweets

def extract_mentions(string):
    hashtags = re.findall(r"@(\w+)", string)
    
    return hashtags


# Iterates over a dataframe and extracts all hashtags

def find_mentions(dataframe):
    
    h_list = []
    
    for index, row in dataframe.iterrows():
        
        hashtags = extract_mentions(row['tweets_raw'])
        
        h_list.append(hashtags)
    
    return h_list

In [103]:
# Extract hashtags and add them as a separate column

hashtags = find_hashtags(tweets_df)

tweets_df['hashtags'] = hashtags

In [168]:
# Extract mentions and add them as a separate column

mentions = find_mentions(tweets_df)

tweets_df['mentions'] = mentions

In [237]:
def get_sentiment(string):
    text = TextBlob(string)
    
    return text.sentiment

def get_dataframe_sentiments(dataframe):
    
    sentiments = []
    
    for index, row in dataframe.iterrows():
        sentiments.append(get_sentiment(row['tweets_clean']))
        
    return sentiments

In [238]:
sentiments = get_dataframe_sentiments(tweets_df)

In [239]:
tweets_df['sentiment'] = sentiments

### LDA

In [273]:
tweets_df_d = tweets_df.head(n=33099)

In [278]:
tweets_df_r = tweets_df.iloc[33099:]

In [283]:
# Construct corpus from the tweets, grouped by politics

In [251]:
s = tweets_df['tweets_clean']
s.str.cat(sep='. ')

corpus = s

In [282]:
s = tweets_df_d['tweets_clean']
s.str.cat(sep='. ')

corpus_d = s

In [289]:
s = tweets_df_r['tweets_clean']

corpus_r = s.str.cat(sep='. ')

In [312]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in tweets_df_r['tweets_clean']]  

In [313]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [314]:
# Creating the object for LDA model
Lda = gensim.models.ldamodel.LdaModel

In [315]:
# Running and Trainign LDA model on the document term matrix
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=1)

In [316]:
print(ldamodel.print_topics(num_topics=10, num_words=5))

[(0, '0.020*"bill" + 0.020*"act" + 0.015*"house" + 0.012*"h" + 0.011*"today"'), (1, '0.017*"thank" + 0.016*"law" + 0.012*"officer" + 0.010*"enforcement" + 0.010*"community"'), (2, '0.021*"great" + 0.011*"thanks" + 0.010*"today" + 0.010*"forward" + 0.008*"meeting"'), (3, '0.011*"government" + 0.010*"health" + 0.010*"national" + 0.009*"amp" + 0.009*"need"'), (4, '0.018*"student" + 0.016*"school" + 0.016*"great" + 0.012*"today" + 0.011*"office"'), (5, '0.014*"amp" + 0.013*"chairman" + 0.012*"state" + 0.011*"u" + 0.010*"president"'), (6, '0.016*"live" + 0.013*"tune" + 0.012*"watch" + 0.010*"amp" + 0.009*"join"'), (7, '0.043*"tax" + 0.026*"job" + 0.024*"n" + 0.021*"american" + 0.019*"taxreform"'), (8, '0.018*"hearing" + 0.018*"amp" + 0.016*"house" + 0.015*"bill" + 0.013*"today"'), (9, '0.025*"family" + 0.022*"amp" + 0.019*"veteran" + 0.015*"happy" + 0.014*"day"')]


In [300]:
import gensim
from gensim import corpora

In [284]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')

en_stop = get_stop_words('en')

In [298]:
def generate_LDA_model(corpus, num_topics=10):
    
    tokens = tokenizer.tokenize(corpus)
    
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    texts = [p_stemmer.stem(i) for i in stopped_tokens]
    
    dictionary = corpora.Dictionary(texts)
    
    new_corpus = [dictionary.doc2bow(text) for text in texts]
    
    ldamodel = gensim.models.ldamodel.LdaModel(new_corpus, num_topics=num_topics, id2word = dictionary, passes=20)
    
    
    return ldamodel
    
    

In [286]:
def print_topics(ldaModel, num_topics=10):
    
    print(ldamodel.print_topics(num_topics=num_topics, num_words=6))

In [299]:
lda_r = generate_LDA_model(corpus_r)

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

### Initial testing and model validation

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = tweets_df.drop(['politics', 'tweets_clean', 'tweets_raw'])
y = tweets_df['politics']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)